In [1]:
%load_ext autoreload
%autoreload 2

# testing the handler

In [2]:
from retrieva.handler import RagHandler
from dotenv import load_dotenv
import os
from retrieva import ROOT_PATH
# used in dev; in production pass the env variable to the containers
load_dotenv(os.path.join(ROOT_PATH, ".env"))

/home/peterdays/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
from IPython.display import Markdown, display, clear_output

# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

In [4]:
from retrieva.data import add_root

In [7]:
rag_handler = RagHandler(
    index_name="SageMakerDocs",
    weaviate_url=os.environ["WEAVIATE_URL"],
    data_path=add_root(os.environ["DATA_FOLDER_PATH"]),
    cloud_based=False
)

2024-04-24 03:09:58,170 | INFO | handler - __init__() 
>>>> Using weaviate db at http://localhost:10080

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


/home/peterdays/miniconda3/envs/retrieva/lib/python3.10/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.5.6.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
Use pytorch device_name: cuda
2024-04-24 03:10:07,770 | INFO | handler - __init__() 
>>>> Loading SageMakerDocs...



In [9]:
# testing
query = "how can I solve a RL aws problem with sagemaker?"
# query = "do you know anything about sagemkaer toolkit?"
# query = "am I an OVNI?"
resp_stream = rag_handler.user_prompt_streaming(query)

In [10]:
sentence = ""
for text in resp_stream.response_gen:
    # return the texts as they arrive.
    clear_output()
    sentence += text
    print(sentence)

Based on the provided context information, it seems that SageMaker is a machine learning service offered by AWS (Amazon Web Services). It provides tools for training and deploying machine learning models. In your query about solving a reinforcement learning (RL) problem with SageMaker, I can suggest the following steps:

1. Check if SageMaker supports RL algorithms. The documentation provided in the context information doesn't mention RL explicitly, but it does list various machine learning tasks that can be performed using SageMaker, such as image and text classification, object detection, and regression. You may want to check the official SageMaker documentation for more details on supported algorithms and workflows.

2. If SageMaker supports RL, you'll need to create a new SageMaker project and follow the steps outlined in the documentation to set up your environment, including installing required dependencies and configuring your training script.

3. You can use the SageMaker Train

In [11]:
prompts_dict = rag_handler.engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>